In [1]:
import time
import functools
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F

DEVICE = 'cpu'

In [2]:
def plot(board, hanle=None, sleep_secs=0.8):
    """ display board """
    board = board.detach().cpu()
    s_board = np.zeros(board.shape, dtype='U')
    s_booard[board == +1] = '○'
    s_booard[board == -1] = '×'
    df = pd.DataFrame(s_board.squeeze())
    if handle is None:
        handle = display(df, display_id=True)
    else:
        handle.update(df)
    time.sleep(sleep_secs)
    return handle

In [3]:
FILTERS = torch.tensor([
    # horizon
    [[[0,0,0],
      [1,1,1],
      [0,0,0]]],
    # vertical
    [[[0,1,0],
      [0,1,0],
      [0,1,0]]],
    # to topleft
    [[[1,0,0],
      [0,1,0],
      [0,0,1]]],
    # to topright
    [[[0,0,1],
      [0,1,0],
      [1,0,0]]],
], dtype=torch.float32, device=DEVICE)

In [4]:
def next_move(board, player, nv, out=None):
    assert board.numel() == 9
    assert player in (-1, +1)
    assert nv in range(board.numel())
    if board.flatten()[nv] != 0:
        return -player
    out = board.detach().clone() if out is None else out
    out.flatten()[nv] = player
    n_match = F.conv2d(out.view(1,1,3,3), FILTERS, stride=1, padding=1)
    mask = n_match.abs() == 3
    state = n_match[mask].sign().sum().clamp(-1,1)
    return state.detach